In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [2]:
df = pd.read_pickle('features/advanced_features.pkl')
df.head()

,Feature,Class
0,"[-620.25830078125, 61.49414825439453, -15.2883...",0.0
1,"[-563.337890625, 18.238452911376953, -2.340278...",1.0
2,"[-736.4028930664062, 55.81863784790039, 9.0535...",0.0
3,"[-278.8795471191406, 100.88816833496094, -25.8...",1.0
4,"[-1131.37109375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0


In [3]:
X = np.array(df.Feature.to_list())
y = np.array(df.Class.to_list())

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [5]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(43, 1)),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [6]:
%%capture --no-stdout
import tensorflow_addons as tfa
# ADDRESS IMBALANCE:
from sklearn.utils.class_weight import compute_class_weight

In [7]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict

{0: 0.9513942847276181, 1: 1.0538395333269581}

In [8]:
tqdm_callback = tfa.callbacks.TQDMProgressBar()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, class_weight=class_weight_dict, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[tqdm_callback])

Training:   0%|                                         0/10 ETA: ?s,  ?epochs/s

Epoch 1/10


2024-10-27 19:25:41.284252: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


0/689                                                                 ETA: ?s - 

Epoch 1/10
689/689 [==============================] - 18s 23ms/step - loss: 0.5281 - accuracy: 0.7255 - val_loss: 0.4534 - val_accuracy: 0.7936
Epoch 2/10


...

Epoch 8/10
689/689 [==============================] - 16s 23ms/step - loss: 0.3320 - accuracy: 0.8509 - val_loss: 0.3203 - val_accuracy: 0.8595
Epoch 9/10


0/689                                                                 ETA: ?s - 

Epoch 9/10
689/689 [==============================] - 16s 23ms/step - loss: 0.3273 - accuracy: 0.8512 - val_loss: 0.3248 - val_accuracy: 0.8586
Epoch 10/10


0/689                                                                 ETA: ?s - 

Epoch 10/10
689/689 [==============================] - 15s 22ms/step - loss: 0.3171 - accuracy: 0.8571 - val_loss: 0.3120 - val_accuracy: 0.8633


In [9]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
test_accuracy

173/173 [==============================] - 1s 5ms/step - loss: 0.3120 - accuracy: 0.8633


0.863339364528656

In [10]:
model.save('./saved_models/cnn_adnv.keras')